# CIFAR-10 Linear Model

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [ ]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model , Sequential
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [ ]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

## Importation du dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [ ]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [ ]:
LOG_DIR = os.path.join("logs") 
MODEL_DIR = os.path.join("models") 

## Fixer les seeds

In [ ]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# Modèle ResNets

##### Learning rate et momentum

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = [0.5,0.1, 0.01, 0.001]
momentums = [0.25, 0.5, 0.9]

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32 for x in range(1, num_layer+1)]

        resnet = ResNet(num_layer, filters_by_layers)
        resnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
        RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
        resnet.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
                       ) 
        resnet.save(RESNET_MODEL)

##### Fonction d'activation 
Tanh Relu Sigmoid

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums =  0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=tanh,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=sigmoid,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

##### Optimizer 
SGD ADAM RMSprop

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = Adam
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = RMSprop
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=RMSprop(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

#### Normalisation

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax,kernel_regularizer=l2(0.5), bias_regularizer=l2(0.5))(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.5")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.5.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

In [ ]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.01")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.01.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)